In [1]:
from fl.preprocessing import preprocess_force_magnitude
import tensorflow as tf 
from tensorflow.keras.models import load_model

model = load_model("../models/force_prediction_model.h5")

## ***>> Assurez-vous d'avoir tensorflow version 2.15***
`pip install tensorflow==2.15`

# Challenge 4 : Du poison [3/2]

modèle : un réseau de neurone capable de prédire la puissance (vitesse) d'un voilier, en se basant sur l'accélération en x, y et z. 

Une étape de préparation des données est effectuée pour calculer l'amplitude (la norme au carré) et pour enlever les données inutiles. Je vous laisse les deux versions à titre indicatif, cela ne devrait pas être important pour résoudre le challenge. 

C'est un modèle de **régression**, c'est-à-dire qu'il ne finit pas sur une sigmoïde, mais sur, rien ! Toutes les valeurs sont possibles en sortie. Même si en pratique, je l'utilise avec une notion de classe, soit 25, soit 50. Si la sortie du modèle est $x$, la classe prédite est :
- 25 si $|25 - x|$ < $|50 - x|$
- 50 sinon

**Le but du challenge est de modifier deux poids, pour que le modèle prédise toutes les classes 25 en 50 et inversement.** 

Vous avez pour cela quatre exemples de données dans `data/example_force_...`. Bonne chance !

In [2]:
from fl.preprocessing import preprocess_force_magnitude
import tensorflow as tf 
from tensorflow.keras.models import load_model

model = load_model("../models/force_prediction_model.h5")

examples = ["25a", "25b", "50a", "50b"]
values = {example: tf.convert_to_tensor(preprocess_force_magnitude(f"../data/example_force_{example}.csv").to_numpy()[:, 0].reshape(1, 50)) for example in examples}
predictions = {example: model.predict(values[example])[0][0] for example in examples}
predictions

1/1 [==============================] - 0s 18ms/step


{'25a': 24.904825, '25b': 25.188282, '50a': 55.79935, '50b': 46.48767}

In [268]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import requests
import json

# Données d'accélération (features)
X_train = np.array([[0.5, 0.2, 0.1], [0.7, 0.3, 0.2], [0.6, 0.1, 0.3], [0.8, 0.4, 0.2]])  

# Puissances cibles (labels) 
y_train = np.array([25, 50, 25, 50])

# Normalisation des données d'entrée entre -1 et 1
X_train_norm = (X_train - 0.5) * 2

# Création du modèle
model = Sequential()
model.add(Dense(128, input_dim=3, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))) 
model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dense(1))

model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

# Définition d'un callback pour arrêter l'entraînement si la loss ne s'améliore plus
early_stop = EarlyStopping(monitor='loss', patience=200)

# Entraînement du modèle avec plus d'epochs et le callback
model.fit(X_train_norm, y_train, epochs=20000, verbose=0, callbacks=[early_stop])

# Prédiction avec le modèle entraîné
print("Prédictions du modèle initial:")
predictions = model.predict(X_train_norm)
print(np.round(predictions))

print("Classes prédites:")
print(np.where(abs(predictions - 25) < abs(predictions - 50), 25, 50).flatten())

# Récupération des poids du modèle entraîné
weights = model.get_weights()

# Modification des poids pour inverser les prédictions 25 <-> 50
weights[-1] *= -2  # Inversion et amplification des poids de la couche de sortie
weights[-1][0] += 100  # Ajout d'une constante de 100 au biais de la couche de sortie

# Modification des poids des couches cachées pour amplifier l'effet d'inversion
weights[-2] *= -3  # Inversion et forte amplification des poids de la dernière couche cachée
weights[-4] *= -2.5   # Inversion et amplification des poids de l'avant-dernière couche cachée  
weights[-6] *= 2  # Amplification des poids de la première couche cachée (sans inversion)

model.set_weights(weights)  # Mise à jour des poids du modèle

# Prédiction avec le modèle modifié 
print("Prédictions après modification des poids:")
predictions = model.predict(X_train_norm)
print(np.round(predictions))

# Calcul du seuil optimal de décision sur les données d'entraînement
threshold = (predictions[np.where(y_train == 25)].mean() + predictions[np.where(y_train == 50)].mean()) / 2
print(f"Seuil de décision optimal : {threshold:.2f}")

print("Classes prédites inversées:")
print(np.where(predictions < threshold, 50, 25).flatten())

# Préparation des données pour la requête
data = {
    "position_1": [-1, 0],   # Indice du poids de biais de la couche de sortie
    "value_1": float(weights[-1][0]),  # Nouvelle valeur du poids de biais
    "position_2": [-2, 0, 0],  # Indices du premier poids de la dernière couche cachée
    "value_2": float(weights[-2][0][0])   # Nouvelle valeur du poids  
}

# Conversion des valeurs flottantes en notation scientifique si nécessaire
for key in data:
    if isinstance(data[key], float) and (data[key] > 1e15 or data[key] < -1e15 or np.isnan(data[key])):
        data[key] = format(data[key], '.17g')

URL = "https://du-poison.challenges.404ctf.fr"
print(requests.post(URL + "/challenges/4", json=data).json()["message"])


Prédictions du modèle initial:
1/1 [==============================] - 0s 58ms/step
[[25.]
 [50.]
 [25.]
 [50.]]
Classes prédites:
[25 50 25 50]
Prédictions après modification des poids:
1/1 [==============================] - 0s 18ms/step
[[100.]
 [100.]
 [100.]
 [100.]]
Seuil de décision optimal : 99.61
Classes prédites inversées:
[25 25 25 25]
Bien joué ! Voici le drapeau : 404CTF{d3_p3t1ts_Ch4ng3m3ntS_tR3s_cHA0t1qU3s}


## Récupération du drapeau

In [4]:
import requests as rq

URL = "https://du-poison.challenges.404ctf.fr"
rq.get(URL + "/healthcheck").json()

{'message': 'Statut : en pleine forme !'}

In [73]:
data = {
  "position_1": [
    0,
    0,
    0
  ],
  "value_1": 0.0,
  "position_2": [
    4,
    0
  ],
  "value_2": 0
}
rq.post(URL + "/challenges/4", json=data).json()["message"]


'Raté ! Le modèle a obtenu une précision sur les classes inversée de 0.0, il faut au moins 0.7'